In [1]:
#!pip install -r https://raw.githubusercontent.com/datamllab/automl-in-action-notebooks/master/requirements.txt

!pip install autokeras
!pip install keras-tuner
#!pip install numpy==1.24.1
#!pip install pandas==1.5.3
#!pip install scikit-learn==1.3.2
#!pip install scipy==1.10.0
#!pip install tensorflow==2.12.0
#!pip install matplotlib==3.4.3
#!pip install seaborn==0.11.2
#!pip install pydotplus==2.0.2
#!pip install graphviz==0.17
#!pip install lightgbm==3.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.7/122.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 10.0 MB/s eta 0:00:00


In [4]:
!pip list --format=freeze > requirements.txt

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
print("Training image shape:", x_train.shape)  # (60000, 28, 28)
print("Training label shape:", y_train.shape)  # (60000,)
print("First five training labels:", y_train[:5])  # array([5 0 4 1 9], dtype=uint8)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training image shape: (60000, 28, 28)
Training label shape: (60000,)
First five training labels: [5 0 4 1 9]


In [ ]:
import autokeras as ak

# Initialize the image classifier.
clf = ak.ImageClassifier(max_trials=2)  # It tries two different models.

# Feed the image classifier with training data
# 20% of the data is used as validation data by default for tuning
# the process may run for a bit long time, please try to use GPU
clf.fit(x_train, y_train, epochs=3)  # each model is trained for three epochs


Reloading Tuner from ./image_classifier/tuner0.json

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
resnet            |vanilla           |image_block_1/block_type
True              |True              |image_block_1/normalize
True              |False             |image_block_1/augment
True              |None              |image_block_1/image_augmentation_1/horizontal_flip
True              |None              |image_block_1/image_augmentation_1/vertical_flip
0                 |None              |image_block_1/image_augmentation_1/contrast_factor
0                 |None              |image_block_1/image_augmentation_1/rotation_factor
0.1               |None              |image_block_1/image_augmentation_1/translation_factor
0                 |None              |image_block_1/image_augmentation_1/zoom_factor
False             |None              |image_block_1/res_net_block_1/pretrained
resnet50          |None              |image_block_1/res_net_block_1/versi

In [ ]:
test_loss, test_acc = clf.evaluate(x_test, y_test, verbose=0)
print("Test accuracy: ", test_acc)


In [ ]:
predicted_y = clf.predict(x_test)
print(predicted_y)


In [ ]:
best_model = clf.export_model()
best_model.summary()


In [ ]:
from tensorflow.keras.models import load_model

best_model.save("model_autokeras")

loaded_model = load_model("model_autokeras")  # , custom_objects=ak.CUSTOM_OBJECTS

predicted_y = loaded_model.predict(tf.expand_dims(x_test, -1))
print(predicted_y)

test_loss, test_acc = clf.evaluate(x_test, y_test, verbose=0)
print("Test accuracy: ", test_acc)


In [ ]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

clf = ak.ImageClassifier(
    max_trials=2,
    loss="categorical_crossentropy",
    metrics=["accuracy"],
    objective="val_accuracy",
)

clf.fit(
    x_train,
    y_train,
    validation_split=0.15,
    epochs=3,
    verbose=2,
)


In [ ]:
import keras_tuner


def my_metric(y_true, y_pred):
    correct_labels = tf.cast(y_true == y_pred, tf.float32)
    return tf.reduce_mean(correct_labels, axis=-1)


clf = ak.ImageClassifier(
    seed=42,
    max_trials=2,
    loss="categorical_crossentropy",
    # Wrap the function into a Keras Tuner Objective
    # and pass it to AutoKeras.
    # Direction can be 'min' or 'max'
    # meaning we want to minimize or maximize the metric.
    # 'val_my_metric' is just add a 'val_' prefix
    # to the function name or the metric name.
    objective=keras_tuner.Objective("val_my_metric", direction="max"),
    # Include it as one of the metrics.
    metrics=[my_metric],
)

clf.fit(x_train, y_train, validation_split=0.15, epochs=3)
